Load selected variables into selected_gss_data.csv:

In [1]:
import pandas as pd
#
var_list = ['year', 'age', 'educ', 'relig', 'happy', 'zodiac','polviews', 'degree'] # List of variables you want to save
output_filename = 'selected_gss_data.csv' # Name of the file you want to save the data to
#
modes = ['w','a'] # Has write mode and append mode
phase = 0 # Starts in write mode; after one iteration of loop, switches to append mode
#
for k in range(3): # for each chunk of the data
    url = 'https://github.com/DS3001/project_gss/raw/main/gss_chunk_' + str(1+k) + '.parquet' # Create url to the chunk to be processed
    #print(url) # Check the url is correct
    df = pd.read_parquet(url) # Download this chunk of data
    #print(df.head()) # Visually inspect the first few rows
    df.loc[:,var_list].to_csv(output_filename, # specifies target file to save the chunk to
                              mode=modes[phase], # control write versus append
                              header=var_list, # variable names
                              index=False) # no row index saved
    phase = 1 # Switch from write mode to append mode

Read .csv into memory

In [3]:
df = pd.read_csv('selected_gss_data.csv', low_memory=False)
df.head()

,year,age,educ,relig,happy,zodiac,polviews,degree
0,1972,23.0,16.0,jewish,not too happy,NaN,NaN,bachelor's
1,1972,70.0,10.0,catholic,not too happy,NaN,NaN,less than high school
2,1972,48.0,12.0,protestant,pretty happy,NaN,NaN,high school
3,1972,27.0,17.0,other,not too happy,NaN,NaN,bachelor's
4,1972,61.0,12.0,protestant,pretty happy,NaN,NaN,high school


Only include the years from 2012 to 2022 (2022 is the latest data, as the older data may reflect different trends and that is not the focus of this investigation)

In [5]:
# Remove any entries in table that specify the word 'year' as they represent uncleaned data and only
# specify which column represents which variable (it's data that should be completely ignored as it represents a row
# with all the variables included like at the top of the table)
df = df.loc[df['year'] != 'year']

# Cast year to number
df['year'] = pd.to_numeric(df['year'], errors='coerce')

# Filter out the years only between 2012 and 2022
df = df.loc[(df.year >= 2012) & (df.year <= 2022)]

# Now only includes data from 2012 to 2022
df.head()

,year,age,educ,relig,happy,zodiac,polviews,degree
55089,2012,22.0,16.0,catholic,very happy,libra,"moderate, middle of the road",bachelor's
55090,2012,21.0,12.0,catholic,very happy,aries,slightly conservative,high school
55091,2012,42.0,12.0,protestant,pretty happy,aries,slightly conservative,high school
55092,2012,49.0,13.0,protestant,very happy,capricorn,slightly conservative,high school
55093,2012,70.0,16.0,protestant,very happy,taurus,liberal,bachelor's


See count of NAs in dataframe df:

In [6]:
print("Rows, columns:", df.shape)
df.isna().sum()

Rows, columns: (17303, 8)


year          0
age         572
educ        101
relig       211
happy        72
zodiac      995
polviews    587
degree       31
dtype: int64

Change column names:

In [7]:
df = df.rename(columns = {'educ':'education',
                    'relig':'religion',
                    'polviews':'political_view', 'degree':'degree_obtained'})

Clean religion variable: (by capitalizing/editing any religion type and replacing any NAs with unknown).

In [8]:
# Capitalize religion type
to_replace = {'catholic': 'Catholic', 'protestant': 'Protestant', 'christian': 'Christian', 'none': 'None',
              'orthodox-christian': 'Orthodox-Christian', 'hinduism': 'Hinduism', 'other': 'Other', 'muslim/islam': 'Islam',
              'other eastern religions': 'Other Eastern Religions', 'buddhism': 'Buddhism', 'inter-nondenominational': 'Inter-Nondenominational',
              'native american': 'Native American', 'relig': 'Unknown'}
df['religion'] = df['religion'].map(to_replace)

# Replace any NAs with unknown and
df['religion'].fillna('Unknown', inplace=True) #replace NAs with 'unknown'

# Print uniques
print(df['religion'].unique())

['Catholic' 'Protestant' 'Christian' 'None' 'Orthodox-Christian'
 'Hinduism' 'Unknown' 'Other' 'Islam' 'Other Eastern Religions' 'Buddhism'
 'Inter-Nondenominational' 'Native American']


Clean age variable: (by casting ages to integers and leaving nans as-is in order to graph as it is a numerical type)

In [ ]:
# Typecast age to float/numeric
df['age'] = pd.to_numeric(df['age'],errors='coerce')

# Converts age to integer and removes the .0 at the end
df['age'] = df['age'].apply(lambda x: int(x) if pd.notnull(x) else x)

print(df['age'].unique())

Clean degree variable: (by capitalizing each degree type and replacing NA's with unknowns)

In [ ]:
# Capitalize each degree type
to_replace = {"bachelor's": "Bachelor's", "high school": "High School", "associate/junior college": "Associate/Junior College",
              "less than high school": 'Less Than High School', "graduate": "Graduate", "unknown": 'Unknown', "degree": "Unknown"}
df['degree_obtained'] = df['degree_obtained'].map(to_replace)

# Replace NA's with Unknowns
df['degree_obtained'].fillna('Unknown', inplace=True) #replace NAs with 'unknown'

# Print uniques
print(df['degree_obtained'].unique())

Clean education variable: (by casting to number and as it is a numeric type, keep NA's for graphing purposes)

In [ ]:
# Typecast education to float/numeric
df['education'] = pd.to_numeric(df['education'], errors='coerce')

# Converts age to integer and removes the .0 at the end
df['education'] = df['education'].apply(lambda x: int(x) if pd.notnull(x) else x)

# Print uniques
print(df['education'].unique())

Clean happy variable: (Replace any NA's with 'Unknown')

In [ ]:
# Capitalize and recategorize
to_replace = {'very happy': 'Very Happy', 'pretty happy': 'Moderately Happy', 'not too happy': 'Not Happy'}
df['happy'] = df['happy'].map(to_replace)

# Replace any NA's with Unknown
df['happy'].fillna('Unknown', inplace=True)

# Print uniques
print(df['happy'].unique())

Clean political_view variable: (capitalizing and renaming certain types to be less vague and replacing any NAs with Unknown). Then, creating new variable (political_view_id) that represents how liberal (1) to how conservative (7) a person is.

In [ ]:
# Capitalize/edit naming of political view
to_replace = {'extremely liberal': "Extremely Liberal", 'liberal': 'Liberal', 'slightly liberal': 'Slightly Liberal', 'moderate, middle of the road': 'Moderate',
              "slightly conservative": 'Slightly Conservative', 'conservative': 'Conservative', 'extremely conservative': 'Extremely Conservative'}
df['political_view'] = df['political_view'].map(to_replace)

# Map political view to political view id
to_replace = {'Extremely Liberal': 1, 'Liberal': 2, 'Slightly Liberal': 3, 'Moderate': 4, 'Slightly Conservative': 5,
              'Conservative': 6, 'Extremely Conservative': 7}
df['political_view_id'] = df['political_view'].map(to_replace)

# Replace NAs with 'Unknown' only for political_view (categorical data type) and not political_view_id (numerical data type)
df['political_view'].fillna('Unknown', inplace=True)

# Print uniques
print(df['political_view'].unique())
print(df['political_view_id'].unique())

Final cleaned CSV:

In [14]:
df.to_csv("cleaned_gss_data.csv", encoding = 'utf-8')
with open("cleaned_gss_data.csv", 'r', encoding='utf-8') as file:
    contents = file.read()
    print(contents)

,year,age,education,religion,happy,zodiac,political_view,degree_obtained
55089,2012,22.0,16.0,Catholic,very happy,libra,"moderate, middle of the road",bachelor's
55090,2012,21.0,12.0,Catholic,very happy,aries,slightly conservative,high school
55091,2012,42.0,12.0,Protestant,pretty happy,aries,slightly conservative,high school
55092,2012,49.0,13.0,Protestant,very happy,capricorn,slightly conservative,high school
55093,2012,70.0,16.0,Protestant,very happy,taurus,liberal,bachelor's
55094,2012,50.0,19.0,Protestant,very happy,taurus,"moderate, middle of the road",bachelor's
55095,2012,35.0,15.0,Christian,pretty happy,scorpio,"moderate, middle of the road",associate/junior college
55096,2012,24.0,11.0,Catholic,pretty happy,scorpio,"moderate, middle of the road",less than high school
55097,2012,28.0,9.0,None,not too happy,libra,conservative,less than high school
55098,2012,28.0,17.0,None,pretty happy,taurus,liberal,bachelor's
55099,2012,55.0,10.0,Catholic,very happy,pisces,extremely liberal,l

Download CSV file:

In [15]:
from google.colab import files
files.download('cleaned_gss_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>